In [1]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device} ")

Using device: cuda 


## ***Build `CNN` from skratch.***

In [4]:
data_path = "../09-Hyperparameter-Tuning-Bayesian Search/fashion-mnist_train.csv"

In [5]:
import pandas as pd
df = pd.read_csv(data_path)

In [6]:
df.sample(3)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
39954,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15702,6,0,0,0,0,0,0,0,0,0,...,168,104,0,0,0,0,0,0,0,0
21276,1,0,0,0,0,0,0,0,0,36,...,77,87,63,0,0,0,0,0,0,0


In [7]:
X = df.iloc[:, 1: ].values
y = df.iloc[:, 0].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
## Scaling the features
X_train = X_train/255.0
X_test = X_test/255.0

In [10]:
class MyCustomDataset(Dataset):
    def __init__(self, features, lable):
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28) # batch size, 2d imgage
        self.lable = torch.tensor(lable, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.lable[index]

In [11]:
train_dataset = MyCustomDataset(X_train, y_train)
test_dataset = MyCustomDataset(X_test, y_test)

In [28]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [29]:
class MyCNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()

        # convolution layer - feature extractions
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_features,  out_channels=32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32,  out_channels=64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # pooling layer
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear((64*7*7), 128),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(64, 10)
        )


    def forward(self, x):
        out = self.features(x)
        return self.classifier(out)

In [15]:
epochs = 100
learning_rate = 0.09

In [30]:
model = MyCNN(1)

## move model to GPU.
model = model.to(device)

loss_functions = nn.CrossEntropyLoss()

## optimizer
optimzer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [31]:
## traing loop

for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        ## move dataset into the gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        ## forward
        output = model(batch_features)

        ## loss
        loss = loss_functions(output, batch_labels)

        ## backward
        optimzer.zero_grad()
        loss.backward()

        ## update the weight
        optimzer.step()

        total_epoch_loss = total_epoch_loss + loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 0.5468454636931419
Epoch: 2 , Loss: 0.3647001503109932
Epoch: 3 , Loss: 0.31545230355858805
Epoch: 4 , Loss: 0.2803340052862962
Epoch: 5 , Loss: 0.2632947616676489
Epoch: 6 , Loss: 0.2387075697084268
Epoch: 7 , Loss: 0.22320295894145967
Epoch: 8 , Loss: 0.2088747838884592
Epoch: 9 , Loss: 0.1986721673111121
Epoch: 10 , Loss: 0.18442202412088712
Epoch: 11 , Loss: 0.17523626909404993
Epoch: 12 , Loss: 0.16846043904870747
Epoch: 13 , Loss: 0.15754346728821594
Epoch: 14 , Loss: 0.15456151494383813
Epoch: 15 , Loss: 0.1401537623355786
Epoch: 16 , Loss: 0.13764435090000432
Epoch: 17 , Loss: 0.13362906381984552
Epoch: 18 , Loss: 0.12498971604183316
Epoch: 19 , Loss: 0.12161196363096435
Epoch: 20 , Loss: 0.11664708040530483
Epoch: 21 , Loss: 0.10930055287169914
Epoch: 22 , Loss: 0.10456259138509631
Epoch: 23 , Loss: 0.10226884293928742
Epoch: 24 , Loss: 0.10099052229337394
Epoch: 25 , Loss: 0.09472405068390072
Epoch: 26 , Loss: 0.09268018964553873
Epoch: 27 , Loss: 0.086263292

In [32]:
model.eval()

MyCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [33]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)
    # print(outputs[0])

    _, predicted = torch.max(outputs, 1)
    # print(predicted)

    total = total + batch_labels.shape[0]
    # print((predicted == batch_labels).sum().item())

    correct = correct + (predicted == batch_labels).sum().item()
    # break

print(correct/total)


0.9214166666666667
